# 2.3 Implicit Intent

W tej aplikacji wykorzystamy mechanizm intentów domniemanych - wyślemy żądanie obłużenia adresu **url** strony domowej naszego wydziału. W pierwszej kolejności przygotowujemy layout.

```xml
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".MainActivity">

    <Button
        android:id="@+id/homeButton"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="Open Homepage"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintLeft_toLeftOf="parent"
        app:layout_constraintRight_toRightOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

</androidx.constraintlayout.widget.ConstraintLayout>
```

Dodajmy obsługę przycisku

```java
    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        setContentView(R.layout.activity_main);

        Button button = findViewById(R.id.homeButton);
        button.setOnClickListener(view -> {
            
        });
    }
```

Zdefiniujmy `String` przechowujący adres url

```java
String url = "http://wfia.uni.wroc.pl/";
```

Następnie tworzymy obiekt `Uri` (łańcuch znaków zapisany zgodnie z określoną składnią) do którego przekazujemy adres.

```java
Uri uri = Uri.parse(url);
```

W kolejnym kroku tworzę `Intent`, który przyjmuje dwa argumenty. Pierwszym jest rodzaj akcji który chcemy wykonać, drugim dane które przekażemy do wyselekcjonowanej aktywności.

```java
Intent intent = new Intent(Intent.ACTION_VIEW, uri);
```

Do `Intent` dodajmy odpowiednią kategorię - naszą kategorią jest `BROWSABLE`

```java
intent.addCategory(Intent.CATEGORY_BROWSABLE);
```

Kolejnym elementem jest sprawdzenie czy aktywność pozwalająca obsłużyć żądaną akcję istnieje na urządzeniu

```java
if (intent.resolveActivity(getPackageManager()) != null)
```

Metoda `resolveActiviy` zwraca `null` gdy taka aktywność nie istnieje. Listę wszystkich aktywności przechowuje `PackageManager` do instancji którego możemy się dostać wywołując metodę `getPackageManager`.

Od **Android 11 (API 30)** w tym miejscu dostaniemy *warning* mówiący o użyciu tagów `<queries>`. Jest to związane z wprowadzonym od API 30 filtrowaniem widoczności pakietów - aby ograniczyć widoczność wszystkich pakietów (aplikacji zainstalowanych na urządzeniu). Niektóre aplikacje są widoczne automatycznie, aby dostać się do innych należy w naszej aplikacji określić jawnie jakie informacje są potrzebne.

Możemy to zrobić w `AndroidManifest.xml`

```xml
<?xml version="1.0" encoding="utf-8"?>
<manifest xmlns:android="http://schemas.android.com/apk/res/android"
    package="pl.edu.uwr.pum.implicitintentjava">

    <queries>
        <intent>
            <action android:name="android.intent.action.VIEW" />
            <category android:name="android.intent.category.BROWSABLE" />
            <data android:scheme="https" />
        </intent>
    </queries>
    ...
```

Tutaj musimy sprecyzować trzy rzeczy
- `action` - jaką akcję chcemy wykonać - odpowiada akcji utworzonego `Intent`
- `category` - kategorię `Intent` - tutaj jest to `BROWSABLE`. 

W aplikacji mamy również dostęp do filtrów intentów, który możemy odnaleźć przy definicji naszejj aktywności głównej w plioku `AndroidManifest`

```xml
<intent-filter>
    <action android:name="android.intent.action.MAIN" />

    <category android:name="android.intent.category.LAUNCHER" />
</intent-filter>
```

To właśnie tutaj możemy określić kategorię i akcje jakie nasza aplikacja może obsłużyć - ta informacja trafia do `PackageManagera`. Jeżeli inna aplikacja zażąda wykonania akcji, którą nasza aplikacja jest w stanie obsłużyć - zostanie ona otwarta gdy jest jedyną aplikacją przypisaną do tej konkretnej akcji, lub zostanie pokazana w selektorze aplikacji wraz ze wszystkimi innymi mogącymi obsłużyć tą akcję.
- `data` - schemat danych który będziemy przekazywać.

Pozostaje już tylko włączyć aktywność wywołując metodą `startActivity`.

Pełny kod metody `onCreate`

```java
    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        setContentView(R.layout.activity_main);

        Button button = findViewById(R.id.homeButton);
        button.setOnClickListener(view -> {
            String url = "http://wfia.uni.wroc.pl/";
            Uri uri = Uri.parse(url);

            Intent intent = new Intent(Intent.ACTION_VIEW, uri);
            intent.addCategory(Intent.CATEGORY_BROWSABLE);

            if (intent.resolveActivity(getPackageManager()) != null)
                startActivity(intent);
        });
    }
```

Możemy przetestować aplikację - po naciśnięciu przycisku powinna włączyć się domyślna przeglądarka lub selektor aplikacji.